Lambda School Data Science

*Unit 2, Sprint 3, Module 2*

---


# Wrangle ML datasets 
- Explore tabular data for supervised machine learning
- Join relational data for supervised machine learning

# Explore tabular data for superviesd machine learning 

In [37]:
# data analysis and wrangling
import pandas as pd
import numpy as np

# visualization
import matplotlib.pyplot as plt
%matplotlib inline

# encoders
from category_encoders import OneHotEncoder, OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

#metrics
from sklearn.metrics import accuracy_score, precision_score, recall_score, classification_report, roc_auc_score

#pipeline
from sklearn.pipeline import make_pipeline

# machine learning
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

In [ ]:
# Download data
import requests

def download(url):
    filename = url.split('/')[-1]
    print(f'Downloading {url}')
    r = requests.get(url)
    with open(filename, 'wb') as f:
        f.write(r.content)
    print(f'Downloaded {filename}')

download('https://lambdaschool-ds-instruction.s3.amazonaws.com/datasets%3Ainstacart_2017_05_01.tar.gz')

In [ ]:
# Uncompress data
import tarfile
tarfile.open('datasets%3Ainstacart_2017_05_01.tar.gz').extractall()

In [2]:
# Change directory to where the data was uncompressed
%cd instacart_2017_05_01

/Users/sisichen/Desktop/Unit2-sprint3-ds25/module2-wrangle-ml-datasets/instacart_2017_05_01


In [3]:
# Print the csv filenames
from glob import glob
for filename in glob('*.csv'):
    print(filename.split('.')[0] + " = pd.read_csv('"+ filename +"')")

products = pd.read_csv('products.csv')
orders = pd.read_csv('orders.csv')
order_products__train = pd.read_csv('order_products__train.csv')
departments = pd.read_csv('departments.csv')
aisles = pd.read_csv('aisles.csv')
order_products__prior = pd.read_csv('order_products__prior.csv')


In [4]:
# products = pd.read_csv('products.csv')
# orders = pd.read_csv('orders.csv')
# order_products__train = pd.read_csv('order_products__train.csv')
# departments = pd.read_csv('departments.csv')
# aisles = pd.read_csv('aisles.csv')
# order_products__prior = pd.read_csv('order_products__prior.csv')


In [5]:
import pandas as pd
# For each csv file, look at its shape & head
df_orders = pd.read_csv('orders.csv')
df_aisles = pd.read_csv('aisles.csv')
df_op_train = pd.read_csv('order_products__train.csv')
df_op_prior = pd.read_csv('order_products__prior.csv')
df_products = pd.read_csv('products.csv')
df_departments = pd.read_csv('departments.csv')


In [6]:
dfs = [df_orders, df_aisles, df_op_train, df_op_prior, df_products,df_departments]
for df in dfs:
    print(df.shape)
    display(df.head())
    print('/n')

(3421083, 7)


,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


/n
(134, 2)


,aisle_id,aisle
0,1,prepared soups salads
1,2,specialty cheeses
2,3,energy granola bars
3,4,instant foods
4,5,marinades meat preparation


/n
(1384617, 4)


...

/n
(49688, 4)


,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


/n
(21, 2)


,department_id,department
0,1,frozen
1,2,other
2,3,bakery
3,4,produce
4,5,alcohol


/n


 **"Will customers reorder this one product?"** (Binary classification)

Which product? How about **the most frequently ordered product?**

### Questions:

- What is the most frequently ordered product?
- How often is this product included in a customer's next order?
- Find all the order_ids that purchased bananas

### What was the most frequently ordered product?

In [7]:
df_op_prior['product_id'].value_counts()

24852    472565
13176    379450
21137    264683
21903    241921
47209    213584
          ...  
37660         1
42235         1
31333         1
3117          1
10806         1
Name: product_id, Length: 49677, dtype: int64

In [8]:
df_op_train['product_id'].value_counts()

24852    18726
13176    15480
21137    10894
21903     9784
47626     8135
         ...  
22373        1
25364        1
26467        1
41652        1
27193        1
Name: product_id, Length: 39123, dtype: int64

In [9]:
df_products[df_products['product_id']==24852]

,product_id,product_name,aisle_id,department_id
24851,24852,Banana,24,4


### How often are bananas included in a customer's next order?

There are [three sets of data](https://gist.github.com/jeremystan/c3b39d947d9b88b3ccff3147dbcf6c6b):

> "prior": orders prior to that users most recent order (3.2m orders)  
"train": training data supplied to participants (131k orders)  
"test": test data reserved for machine learning competitions (75k orders)

In [10]:
len(df_op_train.order_id.unique())

131209

In [11]:
df_op_train[df_op_train['product_id']==24852]

,order_id,product_id,add_to_cart_order,reordered
115,226,24852,2,0
156,473,24852,2,0
196,878,24852,2,1
272,1042,24852,1,1
297,1139,24852,1,1
...,...,...,...,...
1383894,3419531,24852,2,1
1383930,3419542,24852,6,0
1383960,3419629,24852,5,1
1384157,3420088,24852,9,1


In [12]:
len(df_op_train[df_op_train['product_id']==24852])/len(df_op_train.order_id.unique())

0.14271886837030998

### Find all the order_ids that purchased bananas

In [13]:
banana_orders_train = df_op_train[df_op_train['product_id']==24852]['order_id']
banana_orders_prior = df_op_prior[df_op_prior['product_id']==24852]['order_id']

In [15]:
full_list_banana_orders = pd.concat([banana_orders_train, banana_orders_prior]).tolist()

### Create the target variable

In [ ]:
# Method 1: time consuming

In [16]:
df_orders['bananas_or_not'] = df_orders['order_id'].apply(lambda x: x in full_list_banana_orders)

KeyboardInterrupt: 

In [ ]:
# Method 2: use the merge to add the target variable to df_orders

In [19]:
banana_orders_df = pd.DataFrame({'banana_order_id':full_list_banana_orders, 'banana_order_or_not':[1]*len(full_list_banana_orders) })

In [20]:
banana_orders_df

,banana_order_id,banana_order_or_not
0,226,1
1,473,1
2,878,1
3,1042,1
4,1139,1
...,...,...
491286,3421027,1
491287,3421030,1
491288,3421038,1
491289,3421078,1


In [27]:
orders_with_target = df_orders.merge(banana_orders_df, left_on='order_id', right_on='banana_order_id', how='left')

In [28]:
orders_with_target['banana_order_or_not'] = orders_bananas['banana_order_or_not'].fillna(0)

In [29]:
orders_with_target

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,banana_order_id,banana_order_or_not
0,2539329,1,prior,1,2,8,NaN,NaN,0.0
1,2398795,1,prior,2,3,7,15.0,NaN,0.0
2,473747,1,prior,3,3,12,21.0,NaN,0.0
3,2254736,1,prior,4,4,7,29.0,NaN,0.0
4,431534,1,prior,5,4,15,28.0,NaN,0.0
...,...,...,...,...,...,...,...,...,...
3421078,2266710,206209,prior,10,5,18,29.0,2266710.0,1.0
3421079,1854736,206209,prior,11,4,10,30.0,1854736.0,1.0
3421080,626363,206209,prior,12,1,12,18.0,626363.0,1.0
3421081,2977660,206209,prior,13,1,12,7.0,2977660.0,1.0


### II. Split dataset

In [33]:
X = orders_with_target[['order_dow', 'order_hour_of_day', 'days_since_prior_order']]
y = orders_with_target['banana_order_or_not']
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8)

###  III. Set the baseline

In [36]:
print('the baseline roc_auc_score is 0.5!')

the baseline roc_auc_score is 0.5!


### IV. Create a model

In [38]:
# Model 1: Logistic Regression Model
model_lr = make_pipeline(
    OneHotEncoder(use_cat_names=True),
    SimpleImputer(strategy='most_frequent'),
    StandardScaler(),
    LogisticRegression()
)

model_lr.fit(X_train, y_train);

/Users/sisichen/.local/share/virtualenvs/Unit2-sprint3-ds25-B8QTsEAs/lib/python3.8/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


In [ ]:
# Model 2: Random Forest
model_rf = make_pipeline(
    OrdinalEncoder(),
    SimpleImputer(),
    RandomForestClassifier(n_estimators=200)
)

model_rf.fit(X_train, y_train);

### V. Check the metrics

In [ ]:
## Logistic Regression
print("logistic regession model roc auc score for test is", roc_auc_score(y_test, model_lr.predict_proba(X_test)[:,1]))
# Random Forest
print("random forest model roc auc score for test is", roc_auc_score(y_test, model_rf.predict_proba(X_test)[:,1]))

In [ ]:
lr = plot_roc_curve(model_lr, X_test, y_test, label='Logistic')
rf = plot_roc_curve(model_rf, X_test, y_test, ax=lr.ax_, label='RandomForest')
plt.plot([(0,0), (1,1)], color='grey', linestyle='--')
plt.legend();